# Effortless QA Application Development with Amazon Bedrock and Haystack

*Notebook by [Bilge Yucel](https://www.linkedin.com/in/bilge-yucel/)*

[Amazon Bedrock](https://aws.amazon.com/bedrock/) is a fully managed service that provides high-performing foundation models from leading AI startups and Amazon through a single API. You can choose from various foundation models to find the one best suited for your use case.

In this notebook, we'll go through the process of **creating a generative question answering application** tailored for PDF files using the newly added [Amazon Bedrock integration](https://haystack.deepset.ai/integrations/amazon-bedrock) with [Haystack](https://github.com/deepset-ai/haystack) and [OpenSearch](https://haystack.deepset.ai/integrations/opensearch-document-store) to store our documents efficiently. The demo will illustrate the step-by-step development of a QA application designed specifically for the Bedrock documentation, demonstrating the power of Bedrock in the process 🚀

## Setup the Development Environment

### Install dependencies

In [ ]:
%%bash

pip install opensearch-haystack amazon-bedrock-haystack pypdf

### Download Files

For this application, we'll use the user guide of Amazon Bedrock. Amazon Bedrock provides the [PDF form of its guide](https://docs.aws.amazon.com/pdfs/bedrock/latest/userguide/bedrock-ug.pdf). Run the code to download the PDF to `/content/bedrock-documentation.pdf` directory 👇🏼  

In [2]:
import boto3
from botocore import UNSIGNED
from botocore.config import Config

s3 = boto3.client('s3', config=Config(signature_version=UNSIGNED))
s3.download_file('core-engineering', 'public/blog-posts/bedrock-documentation.pdf', '/content/bedrock-documentation.pdf')

### Initialize an OpenSearch Instance on Colab

[OpenSearch](https://opensearch.org/) is a fully open source search and analytics engine and is compatible with the [Amazon OpenSearch Service](https://docs.aws.amazon.com/opensearch-service/latest/developerguide/what-is.html) that’s helpful if you’d like to deploy, operate, and scale your OpenSearch cluster later on.

Let’s install OpenSearch and start an instance on Colab. For other installation options, check out [OpenSearch documentation](https://opensearch.org/docs/latest/install-and-configure/install-opensearch/index/).

In [ ]:
!wget https://artifacts.opensearch.org/releases/bundle/opensearch/2.11.1/opensearch-2.11.1-linux-x64.tar.gz
!tar -xvf opensearch-2.11.1-linux-x64.tar.gz
!chown -R daemon:daemon opensearch-2.11.1
# disabling security. Be mindful when you want to disable security in production systems
!sudo echo 'plugins.security.disabled: true' >> opensearch-2.11.1/config/opensearch.yml

In [4]:
%%bash --bg
cd opensearch-2.11.1 && sudo -u daemon -- ./bin/opensearch

> OpenSearch needs 30 seconds for a fully started server

In [5]:
import time

time.sleep(30)

### API Keys

To use Amazon Bedrock, you need `aws_access_key_id`, `aws_secret_access_key`, and indicate the `aws_region_name`. Once logged into your account, locate these keys under the IAM user's "Security Credentials" section. For detailed guidance, refer to the documentation on [Managing access keys for IAM users](https://docs.aws.amazon.com/IAM/latest/UserGuide/id_credentials_access-keys.html).

In [ ]:
from getpass import getpass

aws_access_key_id = getpass("aws_access_key_id: ")
aws_secret_access_key = getpass("aws_secret_access_key: ")
aws_region_name = input("aws_region_name: ")

## Building an indexing pipeline

Our indexing pipeline will convert the PDF file into a Haystack Document using [PyPDFToDocument](https://docs.haystack.deepset.ai/v2.0/docs/pypdftodocument) and preprocess it by cleaning and splitting it into chunks before storing them in [OpenSearchDocumentStore](https://docs.haystack.deepset.ai/v2.0/docs/opensearch-document-store).

Let’s run the pipeline below and index our file to our document store:

In [10]:
from pathlib import Path

from haystack import Pipeline
from haystack.components.converters import PyPDFToDocument
from haystack.components.preprocessors import DocumentCleaner, DocumentSplitter
from haystack.components.writers import DocumentWriter
from haystack.document_stores import DuplicatePolicy
from opensearch_haystack import OpenSearchDocumentStore

## Initialize the OpenSearchDocumentStore
document_store = OpenSearchDocumentStore()

## Create pipeline components
converter = PyPDFToDocument()
cleaner = DocumentCleaner()
splitter = DocumentSplitter(split_by="sentence", split_length=10, split_overlap=2)
writer = DocumentWriter(document_store=document_store, policy=DuplicatePolicy.SKIP)

## Add components to the pipeline
indexing_pipeline = Pipeline()
indexing_pipeline.add_component("converter", converter)
indexing_pipeline.add_component("cleaner", cleaner)
indexing_pipeline.add_component("splitter", splitter)
indexing_pipeline.add_component("writer", writer)

## Connect the components to each other
indexing_pipeline.connect("converter", "cleaner")
indexing_pipeline.connect("cleaner", "splitter")
indexing_pipeline.connect("splitter", "writer")

Run the pipeline with the files you want to index:

In [ ]:
indexing_pipeline.run({"converter": {"sources": [Path("/content/bedrock-documentation.pdf")]}})

## Building the query pipeline

Let’s create another pipeline to query our application. In this pipeline, we’ll use [OpenSearchBM25Retriever](https://docs.haystack.deepset.ai/v2.0/docs/opensearchbm25retriever) to retrieve relevant information from the OpenSearchDocumentStore and an Amazon Titan model `amazon.titan-text-express-v1` to generate answers with [AmazonBedrockGenerator](https://docs.haystack.deepset.ai/v2.0/docs/amazonbedrockgenerator). You can select and test different models using the dropdown on right.

Then, we’ll define a prompt based on our task, in this case, to generate answers based on the given context and connect these three components to each other to finalize the pipeline.


In [18]:
from haystack.components.builders import PromptBuilder
from haystack.pipeline import Pipeline
from amazon_bedrock_haystack.generators.amazon_bedrock import AmazonBedrockGenerator
from opensearch_haystack import OpenSearchBM25Retriever

## Create pipeline components
retriever = OpenSearchBM25Retriever(document_store=document_store, top_k=15)

## Initialize the AmazonBedrockGenerator with an Amazon Bedrock model
bedrock_model = 'amazon.titan-text-express-v1' # @param ["amazon.titan-text-express-v1", "amazon.titan-text-lite-v1", "anthropic.claude-instant-v1", "anthropic.claude-v1", "anthropic.claude-v2","anthropic.claude-v2:1", "meta.llama2-13b-chat-v1", "meta.llama2-70b-chat-v1", "ai21.j2-mid-v1", "ai21.j2-ultra-v1"]
generator = AmazonBedrockGenerator(model_name=bedrock_model,
                                   aws_access_key_id=aws_access_key_id,
                                   aws_secret_access_key=aws_secret_access_key,
                                   aws_region_name=aws_region_name,
                                   max_length=500)
template = """
{% for document in documents %}
    {{ document.content }}
{% endfor %}

Please answer the question based on the given information from Amazon Bedrock documentation.

{{question}}
"""
prompt_builder = PromptBuilder(template=template)

## Add components to the pipeline
rag_pipeline = Pipeline()
rag_pipeline.add_component("retriever", retriever)
rag_pipeline.add_component("prompt_builder", prompt_builder)
rag_pipeline.add_component("llm", generator)

## Connect the components to each other
rag_pipeline.connect("retriever", "prompt_builder.documents")
rag_pipeline.connect("prompt_builder", "llm")

Ask your question and learn about the Amazon Bedrock service using Amazon Bedrock models!

In [19]:
question = "What is Amazon Bedrock?"
response = rag_pipeline.run({"retriever": {"query": question}, "prompt_builder": {"question": question}})

print(response["llm"]["replies"][0])

Amazon Bedrock is a fully managed service that makes high-performing foundation models (FMs) from leading AI startups and Amazon available for your use through a uniﬁed API. You can choose from a wide range of foundation models to ﬁnd the model that is best suited for your use case. Amazon Bedrock also oﬀers a broad set of capabilities to build generative AI applications with security, privacy, and responsible AI. Using Amazon Bedrock, you can easily experiment with and evaluate top foundation models for your use cases, privately customize them with your data using techniques such as ﬁne-tuning and Retrieval Augmented Generation (RAG), and build agents that execute tasks using your enterprise systems and data sources.
With Amazon Bedrock's serverless experience, you can get started quickly, privately customize foundation models with your own data, and easily and securely integrate and deploy them into your applications using AWS tools without having to manage any infrastructure.


In [20]:
question = "How can I setup Amazon Bedrock?"
response = rag_pipeline.run({"retriever": {"query": question}, "prompt_builder": {"question": question}})

print(response["llm"]["replies"][0])

To setup Amazon Bedrock, follow these steps:
1. Sign up for an AWS account
2. Create an administrative user
3. Grant programmatic access
4. Console access
5. Model access
6. Set up the Amazon Bedrock API


In [21]:
question = "How can I finetune foundation models?"
response = rag_pipeline.run({"retriever": {"query": question}, "prompt_builder": {"question": question}})

print(response["llm"]["replies"][0])

Amazon Bedrock supports fine-tuning of foundation models through model customization jobs. You can create a model customization job by providing a training dataset that you own. During a model customization job, you can tune the hyperparameters of the model to improve its performance on your specific task or domain. For more information, see Custom models.


In [22]:
question = "How should I form my prompts for Amazon Titan models?"
response = rag_pipeline.run({"retriever": {"query": question}, "prompt_builder": {"question": question}})

print(response["llm"]["replies"][0])

When formulating your prompts for Amazon Titan models, it's important to consider the task or instruction you want the models to perform, the context of the task, demonstration examples, and the input text that you want the models to use in their response. Depending on your use case, the availability of data, and the task, your prompt should combine one or more of these components.

For example, if you're asking the model to summarize a text, you might include the text itself, the task or instruction ("Summarize the text"), demonstration examples ("Example summaries"), and the input text ("The following is text from a restaurant review: \"I finally got to check out Alessandro's Brilliant Pizza and it is now one of my favorite restaurants in Seattle. The dining room has a beautiful view over the Puget Sound but it was surprisingly not crowded. I ordered the fried castelvetrano olives, a spicy Neapolitan-style pizza and a gnocchi dish. The olives were absolutely decadent, and the pizza c

In [23]:
question = "How should I form my prompts for Claude models?"
response = rag_pipeline.run({"retriever": {"query": question}, "prompt_builder": {"question": question}})

print(response["llm"]["replies"][0])

When using Claude models with Amazon Bedrock, it's important to wrap your prompts in a conversational style to get desired responses. The main content of the prompt should be wrapped like this: \n\nHuman: {{Main Content}}\n\nAssistant:. For Claude models, prompts sent via the API must contain \n\nHuman: and \n\nAssistant:.
